In [2]:
import numpy as np
import pickle
# load data from 1mil games and train the model
data = np.load('100k/training_set.npz', allow_pickle=True)
val = np.load('100k/validation_set.npz', allow_pickle=True)

In [3]:
trainX = data['X']
trainY = data['y']

In [4]:
valX = val['X']
valY = val['y']

In [5]:
from board_processing import Boardprocessing

# convert the board to a 1D array
trainX1 = np.array([Boardprocessing(board, "V1").get_board_image().reshape(8,8) for board in trainX])
trainX2 = np.array([Boardprocessing(board, "V2").get_board_image().reshape(8, 8) for board in trainX])


In [6]:
valX1 = np.array([Boardprocessing(board, "V1").get_board_image().reshape(8,8) for board in valX])
valX2 = np.array([Boardprocessing(board, "V2").get_board_image().reshape(8,8) for board in valX])

In [145]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def train1(model, optimizer, loss_fn, train_loader, val_loader, num_epochs):
    train_loss = []
    val_loss = []
    cross_loss = []
    train_acc = []
    val_acc = []

    classes = ('0','1','2','3','4','5','6', '7','8','9',)   

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            # convert labels and preds to integers
            int_labels = torch.round(labels * 10).type(torch.LongTensor)
            int_preds = torch.round(preds.float() / 10).type(torch.LongTensor)
            # compute accuracy
            running_acc += torch.sum(int_preds == int_labels)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_acc / len(train_loader.dataset)
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)

        # Validation phase
        model.eval()
        running_loss = 0.0
        running_acc = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                running_acc += torch.sum(preds == labels)

        epoch_loss = running_loss / len(val_loader.dataset)
        epoch_acc = running_acc / len(val_loader.dataset)
        val_loss.append(epoch_loss)
        val_acc.append(epoch_acc)

        # Print progress
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {epoch_loss:.4f}, Val Acc: {epoch_acc:.4f}')

    # Return the training history
    return train_loss, train_acc, val_loss, val_acc


In [208]:
def train(model, optimizer, criterion, train_loader, val_loader, num_epochs, lr=0.0015, momentum=0.9):
    train_loss = []
    val_loss = []
    cross_loss = []
    train_acc = []
    val_acc = []

    classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9',)

    optimizer = torch.optim.SGD(model.parameters(), lr=lr , momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        for inputs, labels in train_loader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            # convert labels and preds to integers
            int_labels = torch.round(labels * 10).type(torch.LongTensor)
            int_preds = torch.round(preds.float() / 10).type(torch.LongTensor)
            # compute accuracy
            running_acc += torch.sum(int_preds == int_labels)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_acc / len(train_loader.dataset)
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)

        # Validation phase
        model.eval()
        running_loss = 0.0
        running_acc = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                if torch.cuda.is_available():
                        inputs, labels = inputs.cuda(), labels.cuda()
                        
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                # convert labels and preds to integers
                int_labels = torch.round(labels * 10).type(torch.LongTensor)
                int_preds = torch.round(
                    preds.float() / 10).type(torch.LongTensor)
                # compute accuracy
                running_acc += torch.sum(preds == int_labels)

        epoch_loss_v = running_loss / len(val_loader.dataset)
        epoch_acc_v = running_acc / len(val_loader.dataset)
        val_loss.append(epoch_loss_v)
        val_acc.append(epoch_acc_v)

        # if accuracy is better than 50% save the model
        if epoch_acc_v > 0.5:
            torch.save(model.state_dict(), 'model_50percent.pth')

        # Print progress
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {epoch_loss_v:.4f}, Val Acc: {epoch_acc_v:.4f}')

    # Return the training history
    return train_loss, train_acc, val_loss, val_acc


In [333]:
%load_ext autoreload
%autoreload 2
from chesscnn import ChessCNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#############################################################

V1 Board representation

In [293]:
import torch
from torchsummary import summary
from torchvision import models

X_train = trainX1
X_train_tensor = torch.tensor(X_train).float()
d1 = X_train_tensor[0]


from chesscnn import ChessCNN

model = ChessCNN()

summary(model, (1,8,8))


RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: []

In [334]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

from chesscnn import ChessCNN

# Assume you have numpy arrays 
# X_train, y_train, X_val, y_val
X_train = trainX1
y_train = trainY
X_val = valX1 
y_val = valY

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train).float()

X_val_tensor = torch.tensor(X_val).float()
y_val_tensor = torch.tensor(y_val).float()


# Create PyTorch datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the model, optimizer, and loss function
model = ChessCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

# Train the model
num_epochs = 10
train_loss, train_acc, val_loss, val_acc = train(
    model, optimizer, loss_fn, train_loader, val_loader, num_epochs)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x4 and 128x64)

In [ ]:
import matplotlib.pyplot as plt

"""

"""


train_loss, train_acc, val_loss, val_acc = train(
    model, optimizer, loss_fn, train_loader, val_loader, num_epochs=10)

plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Val Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

plt.plot(train_acc, label='Train Acc')
plt.plot(val_acc, label='Val Acc')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
# save the model
torch.save(model.state_dict(), '1mil/model.pt')

# load the model
model.load_state_dict(torch.load('1mil/model.pt'))


##################################################################

Same data with V2 board representation

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from chesscnn import ChessCNN

import matplotlib.pyplot as plt

Classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

sc = StandardScaler()
trainX2 = sc.fit_transform(trainX2.reshape(-1, 64))
valX2 = sc.transform(valX2.reshape(-1, 64))

trainX2 = trainX2.reshape(-1, 8, 8)
valX2 = valX2.reshape(-1, 8, 8)

trainX2 = torch.from_numpy(trainX2).float()
valX2 = torch.from_numpy(valX2).float()

trainY2 = torch.from_numpy(trainY2).long()
valY2 = torch.from_numpy(valY2).long()

train_dataset2 = TensorDataset(trainX2, trainY2)
val_dataset2 = TensorDataset(valX2, valY2)

train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True)
val_loader2 = DataLoader(val_dataset2, batch_size=32, shuffle=True)

model2 = ChessCNN().cuda()
criterion2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)

train_loss2, train_acc2, val_loss2, val_acc2 = train(
    model2, optimizer2, criterion2, train_loader2, val_loader2, num_epochs=10)

plt.plot(train_loss2, label='Training loss')
plt.plot(val_loss2, label='Validation loss')
plt.legend(frameon=False)

plt.plot(train_acc2, label='Training accuracy')
plt.plot(val_acc2, label='Validation accuracy')
plt.legend(frameon=False)

model2.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader2:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for images, labels in val_loader2:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
